In [1]:
import cv2
import torch
import models
import os
from Dataset import Dataset
from torch.autograd.variable import Variable
import numpy as np

from util_moduls.Utils import get_device

from lib.NCEAverage import NCEAverage
from lib.NCECriterion import NCECriterion
%matplotlib inline
import matplotlib.pyplot as plt

from test import NN, kNN

from scipy import misc

'''
import subprocess

subprocess.call(["ulimit -n 64000"])
'''

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'\nimport subprocess\n\nsubprocess.call(["ulimit -n 64000"])\n'

In [2]:
def resize2d(img, size):
    return (torch.nn.functional.adaptive_avg_pool2d(Variable(img,requires_grad=False), size)).data

In [3]:
low_dim = 128
checkpoint = torch.load('model_best.pth.tar')
model = models.__dict__[checkpoint['arch']](n_frames=6,low_dim=low_dim)
model = model.cuda()
model = torch.nn.DataParallel(model)

/usr/local/lib/python2.7/dist-packages/torch/nn/parallel/data_parallel.py:25: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


In [4]:
state_dict = checkpoint['state_dict']

In [5]:
from collections import OrderedDict

print('epoch: {}'.format(checkpoint['epoch']))

lemniscate = checkpoint['lemniscate']

model.load_state_dict(state_dict)           


epoch: 5


In [6]:
#n_frames = 6
#frame_gap=4
preload_to_mem = False
#keep_memory_free=15
#sliding_window=False

#j = 0

training_file = "/data/dataset/BDDV_Full/full_train_set.hdf5"
validation_file = "/data/dataset/BDDV_Full/full_val_set.hdf5"



In [7]:
train_dataset = Dataset(training_file,preload_to_mem=preload_to_mem)

Loading /data/dataset/BDDV_Full/full_train_set.hdf5
The file exists: True
Loading hdf5 key 0 from 61272
Loading hdf5 key 100 from 61272
Loading hdf5 key 200 from 61272
Loading hdf5 key 300 from 61272
Loading hdf5 key 400 from 61272
Loading hdf5 key 500 from 61272
Loading hdf5 key 600 from 61272
Loading hdf5 key 700 from 61272
Loading hdf5 key 800 from 61272
Loading hdf5 key 900 from 61272
Loading hdf5 key 1000 from 61272
Loading hdf5 key 1100 from 61272
Loading hdf5 key 1200 from 61272
Loading hdf5 key 1300 from 61272
Loading hdf5 key 1400 from 61272
Loading hdf5 key 1500 from 61272
Loading hdf5 key 1600 from 61272
Loading hdf5 key 1700 from 61272
Loading hdf5 key 1800 from 61272
Loading hdf5 key 1900 from 61272
Loading hdf5 key 2000 from 61272
Loading hdf5 key 2100 from 61272
Loading hdf5 key 2200 from 61272
Loading hdf5 key 2300 from 61272
Loading hdf5 key 2400 from 61272
Loading hdf5 key 2500 from 61272
Loading hdf5 key 2600 from 61272
Loading hdf5 key 2700 from 61272
Loading hdf5 k

Loading hdf5 key 24300 from 61272
Loading hdf5 key 24400 from 61272
Loading hdf5 key 24500 from 61272
Loading hdf5 key 24600 from 61272
Loading hdf5 key 24700 from 61272
Loading hdf5 key 24800 from 61272
Loading hdf5 key 24900 from 61272
Loading hdf5 key 25000 from 61272
Loading hdf5 key 25100 from 61272
Loading hdf5 key 25200 from 61272
Loading hdf5 key 25300 from 61272
Loading hdf5 key 25400 from 61272
Loading hdf5 key 25500 from 61272
Loading hdf5 key 25600 from 61272
Loading hdf5 key 25700 from 61272
Loading hdf5 key 25800 from 61272
Loading hdf5 key 25900 from 61272
Loading hdf5 key 26000 from 61272
Loading hdf5 key 26100 from 61272
Loading hdf5 key 26200 from 61272
Loading hdf5 key 26300 from 61272
Loading hdf5 key 26400 from 61272
Loading hdf5 key 26500 from 61272
Loading hdf5 key 26600 from 61272
Loading hdf5 key 26700 from 61272
Loading hdf5 key 26800 from 61272
Loading hdf5 key 26900 from 61272
Loading hdf5 key 27000 from 61272
Loading hdf5 key 27100 from 61272
Loading hdf5 k

Loading hdf5 key 50500 from 61272
Loading hdf5 key 50600 from 61272
Loading hdf5 key 50700 from 61272
Loading hdf5 key 50800 from 61272
Loading hdf5 key 50900 from 61272
Loading hdf5 key 51000 from 61272
Loading hdf5 key 51100 from 61272
Loading hdf5 key 51200 from 61272
Loading hdf5 key 51300 from 61272
Loading hdf5 key 51400 from 61272
Loading hdf5 key 51500 from 61272
Loading hdf5 key 51600 from 61272
Loading hdf5 key 51700 from 61272
Loading hdf5 key 51800 from 61272
Loading hdf5 key 51900 from 61272
Loading hdf5 key 52000 from 61272
Loading hdf5 key 52100 from 61272
Loading hdf5 key 52200 from 61272
Loading hdf5 key 52300 from 61272
Loading hdf5 key 52400 from 61272
Loading hdf5 key 52500 from 61272
Loading hdf5 key 52600 from 61272
Loading hdf5 key 52700 from 61272
Loading hdf5 key 52800 from 61272
Loading hdf5 key 52900 from 61272
Loading hdf5 key 53000 from 61272
Loading hdf5 key 53100 from 61272
Loading hdf5 key 53200 from 61272
Loading hdf5 key 53300 from 61272
Loading hdf5 k

In [8]:
val_dataset = Dataset(validation_file,preload_to_mem=preload_to_mem)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=1, shuffle=False, 
    num_workers=0)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=1, shuffle=False,
    num_workers=0)

Loading /data/dataset/BDDV_Full/full_val_set.hdf5
The file exists: True
Loading hdf5 key 0 from 9048
Loading hdf5 key 100 from 9048
Loading hdf5 key 200 from 9048
Loading hdf5 key 300 from 9048
Loading hdf5 key 400 from 9048
Loading hdf5 key 500 from 9048
Loading hdf5 key 600 from 9048
Loading hdf5 key 700 from 9048
Loading hdf5 key 800 from 9048
Loading hdf5 key 900 from 9048
Loading hdf5 key 1000 from 9048
Loading hdf5 key 1100 from 9048
Loading hdf5 key 1200 from 9048
Loading hdf5 key 1300 from 9048
Loading hdf5 key 1400 from 9048
Loading hdf5 key 1500 from 9048
Loading hdf5 key 1600 from 9048
Loading hdf5 key 1700 from 9048
Loading hdf5 key 1800 from 9048
Loading hdf5 key 1900 from 9048
Loading hdf5 key 2000 from 9048
Loading hdf5 key 2100 from 9048
Loading hdf5 key 2200 from 9048
Loading hdf5 key 2300 from 9048
Loading hdf5 key 2400 from 9048
Loading hdf5 key 2500 from 9048
Loading hdf5 key 2600 from 9048
Loading hdf5 key 2700 from 9048
Loading hdf5 key 2800 from 9048
Loading hdf5

In [10]:
ndata = train_dataset.__len__()
nce_k = 4096
nce_t = .07
nce_m = .5
iter_size = 1

n_frames = 6
gpu=0

criterion = NCECriterion(ndata).to(get_device(gpu))

#for i, (input_imgs, action_probabilities, indices) in enumerate(val_loader):
#
#          
#    indices = indices.to(get_device(gpu))
#
#        
#    #input_imgs = resize2d(input_imgs, (224,224))
#    #imgs_for_display = np.array(input_imgs.cpu())
#    
#    input_imgs = input_imgs[:,0:int((n_frames/2)*3),:,:] #extract the first half of the images
#    action_probabilities = action_probabilities[:,0:int(n_frames/2)] #extract steers first 3 out of 6 
#
#    #cv2.imshow('test',(np.array(input_imgs.cpu())+0.5)[0,0:3,:,:].transpose((1,2,0)))
#    #cv2.waitKey(30)
#    trainFeatures = lemniscate.memory.t()
#    features = model(input_imgs, action_probabilities)
#    output = lemniscate(features, indices)
#    loss = criterion(output, indices) / iter_size
#    
#    dist = torch.mm(features, trainFeatures)
#
#    yd, yi = dist.topk(topk, dim=1, largest=True, sorted=True)
#    candidates = trainLabels.view(1,-1).expand(batchSize, -1)
#    retrieval = torch.gather(candidates, 1, yi)
#
#    retrieval = retrieval.narrow(1, 0, topk).clone()
#    yd = yd.narrow(1, 0, topk)
##    
#    if i > 12:
#        break
#cv2.destroyAllWindows()

In [11]:
import torch
import torch.nn as nn
from collections import defaultdict
import pandas as pd

In [46]:
font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (10,50)
fontScale              = 1
fontColor              = (0,0,255)
lineType               = 2

def write_text(img,text):

    cv2.putText(img,str(text), 
        bottomLeftCornerOfText, 
        font, 
        fontScale,
        fontColor,
        lineType)

    return img

In [47]:
from sklearn.metrics import log_loss

In [ ]:
model.eval()
debug = True

topk = 30 # It became evident that the top 5 NNs are sufficient for the best results
steer_eval = {}

correct = 0.
correct_past = 0.
correct_future = 0.

total = 0
observed_timestep = 3

trainFeatures = lemniscate.memory.t()

trainLabels = torch.LongTensor(train_loader.dataset.train_labels).cuda()

#stat_data = pd.DataFrame(columns=[range(topk)])

with torch.no_grad():

    criterion = nn.BCELoss(reduce=False)
    bce = lambda t1, t2: criterion(t1, t2).mean().flatten()

    for batch_idx, (input_imgs, targets, indexes) in enumerate(val_loader):
        targets = targets.cuda(async=True)
        indexes = indexes.cuda(async=True)
        input_imgs = input_imgs.cuda(async=True)

        batchSize = input_imgs.size(0)
        
        og_targets = targets.clone()

        input_imgs = input_imgs[:,0:int((n_frames/2)*3),:,:] #extract only img 1 through 3
        targets = targets[:,0:int(n_frames/2)] #extract steers first 3 targets

        features = model(input_imgs, targets)

        dist = torch.mm(features, trainFeatures)

        yd, yi = dist.topk(topk, dim=1, largest=True, sorted=True)
        candidates = trainLabels.view(1,-1).expand(batchSize, -1)
        retrieval = torch.gather(candidates, 1, yi)

        retrieval = retrieval.narrow(1, 0, topk).clone()
        yd = yd.narrow(1, 0, topk)

        image_steering_labels = defaultdict()
        
        indexes = np.array(indexes.cpu())
        
        img_steer_values = []
        
        for top_id in range(topk):

            ret_ind = int(retrieval[0, top_id])
            img_steer_lab = train_loader.dataset.__getlabel__(ret_ind)[0]
            img_steer_values.append(img_steer_lab[observed_timestep][2:4].data.cpu().numpy())
            
        avg_left = np.average(np.array(img_steer_values)[:,0])
        avg_right = np.average(np.array(img_steer_values)[:,1])
        
        show_img = (input_imgs[0,0:3,:,:].cpu().numpy()+0.5).transpose((1,2,0))
        
        show_img = write_text(show_img.copy(),(avg_left*avg_right))
        cv2.imshow('test',show_img)
        cv2.waitKey(300)
        
        if batch_idx > 200:
            break
cv2.destroyAllWindows()


In [21]:
bce

<function __main__.<lambda>>